In [2]:
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime


# data / bert import

In [9]:
data = pd.read_csv('../data/raw_postpro.csv') # 오류가나면 추가해주세요 .encoding = 'cp949'
# 컬럼 삭제
df = data.drop(['청구서번호','No.',  '선박입고','완료 여부','리드타임_음수제거','청구량','견적','견적수량','견적화폐','견적단가','발주번호','발주','발주수량','발주금액','미입고 기간','리드타임','창고입고','창고입고수량','입고창고','창고출고','창고출고수량','출고선박','출고운반선','선박입고','선박입고수량','완료 여부'], axis=1)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20517 entries, 0 to 20516
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Subject      20517 non-null  object
 1   Machinery    20517 non-null  object
 2   Assembly     20517 non-null  object
 3   청구품목         20517 non-null  object
 4   Part No.1    20517 non-null  object
 5   Part No.2    20517 non-null  object
 6   key1         20517 non-null  object
 7   key2         20517 non-null  object
 8   발주처          20517 non-null  object
 9   D/T          20517 non-null  object
 10  Control No.  20517 non-null  object
 11  leadtime     20517 non-null  int64 
dtypes: int64(1), object(11)
memory usage: 1.9+ MB


In [11]:
df = df[['청구품목','발주처','Machinery', 'Assembly' , "key1",'key2']]
# 'Machinery', 'Assembly', '청구품목', 'Part No.1', 'Part No.2', 'key1', '발주처'

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20517 entries, 0 to 20516
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   청구품목       20517 non-null  object
 1   발주처        20517 non-null  object
 2   Machinery  20517 non-null  object
 3   Assembly   20517 non-null  object
 4   key1       20517 non-null  object
 5   key2       20517 non-null  object
dtypes: object(6)
memory usage: 961.9+ KB


In [13]:
from sklearn import preprocessing
label_encoders = {}  # 각 열에 대한 LabelEncoder를 저장하기 위한 딕셔너리
columns_to_encode = ['key2']  # 인코딩을 수행할 열의 이름 리스트

for column in columns_to_encode:
    le = preprocessing.LabelEncoder()
    le.fit(df[column])
    label_encoders[column] = le # 딕셔너리에 저장
    df[column+"_encoded"] = le.transform(df[column]) # 새로운 encoding 된 컬럼 추가

In [14]:
df = df.drop(['key2'], axis=1)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20517 entries, 0 to 20516
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   청구품목          20517 non-null  object
 1   발주처           20517 non-null  object
 2   Machinery     20517 non-null  object
 3   Assembly      20517 non-null  object
 4   key1          20517 non-null  object
 5   key2_encoded  20517 non-null  int32 
dtypes: int32(1), object(5)
memory usage: 881.7+ KB


In [17]:
# df_desc = df[[ '청구품목', '발주처']].apply(lambda row: ' '.join(row), axis=1)
df_desc = df[[ '청구품목', '발주처','Machinery', 'Assembly' , "key1"]].apply(lambda row: ' '.join(row), axis=1)

In [18]:
lis = df_desc.values

In [22]:
cnt = 0
for i in lis:
    print(i)
    cnt += 1
    if cnt == 20:
        break

SEAL-O-RING-STOR HAEIN Coporation_Cheonan NO.1 GENERATOR ENGINE 323-6480 LINES GP-FUEL  7.00E-275
OIL COOLER & LINES HAEIN Coporation_Cheonan NO.2 GENERATOR ENGINE GASKET KIT 7.00E-275
WASHER HAEIN Coporation_Cheonan NO.2 GENERATOR ENGINE 285-8374 MANIFOLD GP-EXH  7.00E-275
BOLT-HIGH TEMP HAEIN Coporation_Cheonan NO.1 GENERATOR ENGINE 159-8828 TURBO GP 7.00E-275
SEAL HAEIN Coporation_Cheonan NO.1 GENERATOR ENGINE 2N4727 INSTRUMNT PANEL GP 7.00E-275
CORE CHARGES FOR CYLINDER PACK AS HAEIN Coporation_Cheonan NO.2 GENERATOR ENGINE 8N-6224 PISTON GP-ROD& 7.00E-275
PUMP GP-F TFR-REMAN HAEIN Coporation_Cheonan NO.3 GENERATOR ENGINE 8N-6151 PUMP GP-F TFR  7.00E-275
GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.1 GENERATOR ENGINE 8N-7174 GEAR GP-FRONT  7.00E-275
GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.3 GENERATOR ENGINE 8N-7174 GEAR GP-FRONT  7.00E-275
GEAR-WTR PUMP DR HAEIN Coporation_Cheonan NO.3 GENERATOR ENGINE 8N-7174 GEAR GP-FRONT  7.00E-275
GEAR-WTR PUMP DR HAEIN Coporation_Cheonan 

In [26]:
#사전 학습된 BERT multilingual의 토크나이저 활용
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in lis]
print('tokenized_texts[0]:',tokenized_texts[0])

tokenized_texts[0]: ['SE', '##AL', '-', 'O', '-', 'RI', '##NG', '-', 'ST', '##OR', 'H', '##A', '##E', '##IN', 'Cop', '##oration', '_', 'Che', '##onan', 'NO', '.', '1', 'GE', '##NE', '##RA', '##TO', '##R', 'EN', '##GI', '##NE', '323', '-', '648', '##0', 'L', '##INE', '##S', 'GP', '-', 'F', '##UE', '##L', '7', '.', '00', '##E', '-', '275']


In [28]:
# 한글이 있는 데이터 확인
# tokenized_texts[14]

In [29]:
list_len =[len(i) for i in tokenized_texts]

In [31]:
max(list_len)

107

## 문장을 같은 길이로 패딩

In [32]:
#token들의 max length보다 크게 MAX_LEN을 설정한다. 설정한 MAX_LEN만큼 빈 공간을 0 이 채운다
print('padding')
MAX_LEN = 108 # 최대 긴 문장이 107개 단어로 구성됨
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen = MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]
print('input_ids[0]:',input_ids[0])

padding
input_ids[0]: [ 23056  32002    118    152    118  56658  34065    118  53317  42622
    145  10738  11259  27128  94160 101507    168  44131  36233  49307
    119    122  62997  93280  29990  60493  11273  31278 100075  93280
  28550    118  61485  10929    149  83198  10731  25236    118    143
  62674  11369    128    119  11025  11259    118  23896      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0]


In [33]:
#학습 속도를 높이기 위해 실 데이터가 있는 곳과 padding이 있는곳을 attention에게 알려줌

attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print('attention_masks[0]:',attention_masks[0])

attention_masks[0]: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [39]:
# df

In [37]:
y = df['key2_encoded'].values

In [38]:
y

array([18, 18, 18, ...,  5, 20, 20])

In [40]:
# 순서 뒤섞이지 않도록 random_state를 일정하게 고정.
# test set은 위에서 분리되었기에 , train 과 validation set만 분리
print('split train - val')
train_inputs, test_inputs, train_labels, test_labels = \
train_test_split(input_ids, y, random_state=42, test_size=0.2)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42,test_size=0.2)

split train - val


In [41]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=61)

KeyboardInterrupt
